In [1]:
import numpy as np
import networkx as nx
from ipysigma import Sigma

In [2]:
class Node:
    def __init__(self, time, index):
        self.time = time
        self.index = index
        self.parent = None
        self.children = []

    def adopt(self, child):
        self.children.append(child)
        child.parent = self

In [3]:
class LazyWrightFisher:
    def __init__(self, pop_size):
        self.pop_size = pop_size
        self.nodes = dict()

    def node(self, time, index):
        if (time, index) not in self.nodes:
            self.nodes[(time, index)] = Node(time, index)
        return self.nodes[(time, index)]
    
    def parent(self, time, index):
        node = self.node(time, index)
        if node.parent is not None:
            return node.parent
        p = np.random.randint(0, self.pop_size)
        parent_node = self.node(time-1, p)
        parent_node.adopt(node)
        return parent_node
    
    def branch_length(self, time, indexes):
        ans = 0
        t = time
        nodes = set(self.node(time, index) for index in indexes)

        while len(nodes) > 1:
            ans += len(nodes)
            nodes = set(self.parent(t, node.index) for node in nodes)
            t -= 1
        return ans
    
    def graph_index(self, time, index):
        return time * self.pop_size + index
    
    def graph(self):
        G = nx.DiGraph()
        for node in self.nodes.values():
            node_index = self.graph_index(node.time, node.index)
            G.add_node(node_index, x=node.time, y=node.index)
            for child in node.children:
                child_index = self.graph_index(child.time, child.index)
                G.add_node(child_index, x=child.time, y=child.index)
                G.add_edge(node_index, child_index)
        return G

In [5]:
# test code

lwf = LazyWrightFisher(100)

lwf.branch_length(100, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

G = lwf.graph()

sigma = Sigma(G, hide_info_panel=True, hide_search=True)
sigma

Sigma(nx.DiGraph with 299 nodes and 298 edges)